In [2]:
from pprint import pprint
import json
import os

## Data

### HF Spider
Note that Hugging Face's spider dataset does not work with Picard T5 because it does not serialize the DB schemas.

In [3]:
import datasets

dataset = datasets.load_dataset("spider")

Found cached dataset spider (/home/yjunteng/.cache/huggingface/datasets/spider/spider/1.0.0/4e5143d825a3895451569c8b9b55432b91a4bc2d04d390376c950837f4680daa)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
ds_train = dataset["train"] # To get the training rows
pprint(ds_train[0:1].keys())
pprint(ds_train[0])

### Yale Spider
The original Spider dataset was downloaded from their [homepage](https://yale-lily.github.io/spider).
The 2 important files are described on their [github](https://github.com/taoyds/spider) as follows:
1. **train.json/dev.json**
    - `question`: the natural language question
    - `question_toks`: the natural language question tokens
    - `db_id`: the database id to which this question is addressed.
    - `query`: the SQL query corresponding to the question.
    - `query_toks`: the SQL query tokens corresponding to the question.
    - `sql`: parsed results of this SQL query using process_sql.py. Please refer to parsed_sql_examples.sql in thepreprocess directory for the detailed documentation.
2. **tables.json** contains the schema of all tables
    - `db_id`: database id
    - `table_names_original`: original table names stored in the database.
    - `table_names`: cleaned and normalized table names. We make sure the table names are meaningful. [to be changed]
    - `column_names_original`: original column names stored in the database. Each column looks like: [0, "id"]. 0 is the index of table names in table_names, which is city in this case. "id" is the column name.
    - `column_names`: cleaned and normalized column names. We make sure the column names are meaningful. [to be changed]
    - `column_types`: data type of each column
    - `foreign_keys`: foreign keys in the database. [3, 8] means column indices in the column_names. These two columns are foreign keys of two different tables.
    - `primary_keys`: primary keys in the database. Each number is the index of column_names.

In [4]:
spider_dir_path = "./data/spider"

print(os.listdir(spider_dir_path))
print()

train_json_filename = "train_spider.json"
print(f"---{train_json_filename}---")
with open(os.path.join(spider_dir_path, train_json_filename)) as f:
    train_json = json.load(f)
print(train_json[0].keys())
print()

tables_json_filename = "tables.json"
print(f"---{tables_json_filename}---")
with open(os.path.join(spider_dir_path, tables_json_filename)) as f:
    tables_json = json.load(f)
print(tables_json[0].keys())
print()

dev_json_filename = "dev.json"
print(f"---{dev_json_filename}---")
with open(os.path.join(spider_dir_path, dev_json_filename)) as f:
    dev_json = json.load(f)
print(dev_json[0].keys())


['train_others.json', 'dev.json', 'README.txt', 'database', '.DS_Store', 'train_spider.json', 'tables.json', 'train_gold.sql', 'dev_gold.sql']

---train_spider.json---
dict_keys(['db_id', 'query', 'query_toks', 'query_toks_no_value', 'question', 'question_toks', 'sql'])

---tables.json---
dict_keys(['column_names', 'column_names_original', 'column_types', 'db_id', 'foreign_keys', 'primary_keys', 'table_names', 'table_names_original'])

---dev.json---
dict_keys(['db_id', 'query', 'query_toks', 'query_toks_no_value', 'question', 'question_toks', 'sql'])


Tables json is a list, so we load in memory as a dictionary (hashtable) for quicker access.

In [5]:
tables_dict_by_db = {}
for table in tables_json:
    tables_dict_by_db[table["db_id"]] = table 
print("Length of both data structures match:", len(tables_json) == len(tables_dict_by_db))

Length of both data structures match: True


Here is an example of one entry.

In [6]:
list(tables_dict_by_db.keys())[99] #movie_1
movie_1 = tables_dict_by_db["movie_1"]
print("Databse:", movie_1["db_id"])
for key in movie_1:
    # Skip db_id since it is a string
    if key == "db_id":
        continue
    print(f"---{key}---")
    item_collated = "|".join(str(item) for item in movie_1[key])
    print(item_collated)


Databse: movie_1
---column_names---
[-1, '*']|[0, 'movie id']|[0, 'title']|[0, 'year']|[0, 'director']|[1, 'reviewer id']|[1, 'name']|[2, 'reviewer id']|[2, 'movie id']|[2, 'rating stars']|[2, 'rating date']
---column_names_original---
[-1, '*']|[0, 'mID']|[0, 'title']|[0, 'year']|[0, 'director']|[1, 'rID']|[1, 'name']|[2, 'rID']|[2, 'mID']|[2, 'stars']|[2, 'ratingDate']
---column_types---
text|number|text|number|text|number|text|number|number|number|time
---foreign_keys---
[7, 5]|[8, 1]
---primary_keys---
1|5
---table_names---
movie|reviewer|rating
---table_names_original---
Movie|Reviewer|Rating


We now try to serialize each database's schema in accordance with Tscholak (who in turn bases it of Shaw)

In [7]:
#TODO find out how to describe fields
delimiter = " | "
def serialize_spider_db(db):

    # First group column names by their table id
    columns = db["column_names"]
    column_strings = {}
    for column in columns:
        table_idx = column[0]
        if table_idx not in column_strings:
            column_strings[table_idx] = [] 
        # Note that the white spaces in column names were replaced with underscores (arbitrarily I suppose)
        column_strings[table_idx].append(column[1].replace(" ","_"))
    
    # Next combine table name with column names
    tables = db["table_names"]
    table_strings = [db["db_id"]]
    for table_idx in range(len(tables)):
        table_name = tables[table_idx]
        columns_serialized = ", ".join(column_strings[table_idx])
        table_serialized = table_name + " : " + columns_serialized
        table_strings.append(table_serialized)
    
    # Lastly combine all serialized table names together with the db id
    schema_serialized = delimiter.join(table_strings)
    return schema_serialized

serialize_spider_db(movie_1) # 'movie_1 | movie : movie id, title, year, director | reviewer : reviewer id, name | rating : reviewer id, movie id, rating stars, rating date'

'movie_1 | movie : movie_id, title, year, director | reviewer : reviewer_id, name | rating : reviewer_id, movie_id, rating_stars, rating_date'

We will no create some utility functions that will help convert the spider dataset into a testing set.

In [8]:
def test_2_preprocessed(test_case):
    db_id = test_case['db_id']
    db_schema = tables_dict_by_db[db_id]
    db_schema_serialized = serialize_spider_db(db_schema)
    question = test_case["question"]
    return delimiter.join([question, db_schema_serialized])

def test_2_answer(test_case):
    return test_case['query']

def convert_to_training(test_cases : list):
    # This takes in a list and outputs two lists
    if not isinstance(test_cases, list):
        test_cases = [test_cases]

    training_set = {
        "input" : [],
        "output": [],
    }

    def helper(test_case):
        training_set["input"].append(test_2_preprocessed(test_case))
        training_set["output"].append(test_2_answer(test_case))

    for test_case in test_cases:
        helper(test_case)

    return training_set


train_example = train_json[0]
print("Example input:", test_2_preprocessed(train_example))
print("Example output:", test_2_answer(train_example))
print("Example training data:")
pprint(convert_to_training(train_example))

Example input: How many heads of the departments are older than 56 ? | department_management | department : department_id, name, creation, ranking, budget_in_billions, num_employees | head : head_id, name, born_state, age | management : department_id, head_id, temporary_acting
Example output: SELECT count(*) FROM head WHERE age  >  56
Example training data:
{'input': ['How many heads of the departments are older than 56 ? | '
           'department_management | department : department_id, name, '
           'creation, ranking, budget_in_billions, num_employees | head : '
           'head_id, name, born_state, age | management : department_id, '
           'head_id, temporary_acting'],
 'output': ['SELECT count(*) FROM head WHERE age  >  56']}


In [9]:
training_set = convert_to_training(train_json)
training_inputs = training_set['input']
training_outputs = training_set['output']
pprint(training_inputs[:2])
pprint(training_outputs[:2])

['How many heads of the departments are older than 56 ? | '
 'department_management | department : department_id, name, creation, ranking, '
 'budget_in_billions, num_employees | head : head_id, name, born_state, age | '
 'management : department_id, head_id, temporary_acting',
 'List the name, born state and age of the heads of departments ordered by '
 'age. | department_management | department : department_id, name, creation, '
 'ranking, budget_in_billions, num_employees | head : head_id, name, '
 'born_state, age | management : department_id, head_id, temporary_acting']
['SELECT count(*) FROM head WHERE age  >  56',
 'SELECT name ,  born_state ,  age FROM head ORDER BY age']


## Model
Loading model into RAM.

### Picard + T5
From Tscholak

In [41]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("tscholak/cxmefzzi")
model = AutoModelForSeq2SeqLM.from_pretrained("tscholak/cxmefzzi")

### Baseline
Try to follow the HF tutorial with tscholak

#### Preprocessing With a Tokenizer
The example sentence was taken from the HF website.

In [54]:
raw_inputs = [
    "How many singers do we have? | concert_singer | stadium : stadium_id, location, name, capacity, highest, lowest, average | singer : singer_id, name, country, song_name, song_release_year, age, is_male | concert : concert_id, concert_name, theme, stadium_id, year | singer_in_concert : concert_id, singer_id",
]
# raw_inputs = map(test_2_preprocessed, train_json[0:10])
inputs = tokenizer(list(raw_inputs), padding=True, return_tensors="pt")
print(inputs["input_ids"].size())

torch.Size([1, 102])


#### Running Inference
The model generates the desired response. It seems that the SQL tokens are generally lowercase.

In [55]:
outputs = model.generate(inputs.input_ids, max_new_tokens=512)
# outputs = model.generate(**inputs, decoder_input_ids=decoder_inputs.input_ids, max_new_tokens=1024)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
response

['concert_singer | select count(*) from singer']

### T5-Small
For practice we will try to re-train T5 using the spider dataset. Picard based itself of the [T5ForConditionalGeneration](https://huggingface.co/docs/transformers/model_doc/t5#transformers.T5ForConditionalGeneration).

In [11]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenzier = AutoTokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [16]:
import torch
torch.cuda.set_device(3)
torch.cuda.current_device()

3

In [83]:
# training
inputs = tokenizer(training_inputs, padding=True, return_tensors="pt")
labels = tokenizer(training_outputs, padding=True, return_tensors="pt")
outputs = model(input_ids=inputs.input_ids, labels=labels.input_ids)
loss = outpus.loss
logits = outputs.logits